In [2]:
# all imports
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd
import random
from matplotlib.patches import Ellipse
import ligo.skymap.plot
from scipy.optimize import minimize
import numdifftools as ndt
from scipy.interpolate import griddata
from matplotlib.tri import Triangulation
from matplotlib.colors import LogNorm
from matplotlib.pyplot import *

In [5]:
# define directories
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/'
scratch_dir = '/pscratch/sd/j/jjpim/'
v_dir = '/pscratch/sd/j/jjpim/massmag/'
hist_dir = '/pscratch/sd/j/jjpim/'

# load updated master table
tf_mastertable = Table.read(data_dir + 'tf_mastertable_massmag_stellar.fits') 

#load SGA table
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')

#load H1 index
h1_table = Table.read('/global/homes/k/kadglass/DESI_SGA/data/SGA-2020_ALFALFA.fits')

#load H1 data
h1_mass = Table.read('/pscratch/sd/j/jjpim/a100.code12.table2.190808.csv')

h1_table[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,ALFALFA_index
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int64
2,SGA-2020 2,PGC1283207,1283207,228.3770865,5.4232017,S?,152.2,0.36307806,0.724436,0.03463229,23.40448,16.976,False,LEDA-20181114,0,PGC1283207,1,True,228.3770865,5.4232017,0.36307806,2283p055,228.3770803831908,5.423191398593787,0.49470574,SB26,158.20142,0.545691,228.37700918822188,5.4232652570544015,10.897086,3.3509698,3.1147978,3.240862,5.902337,6.9126143,7.941369,8.997992,10.073601,11.199986,12.391357,13.561038,14.841172,16.966799,16.108246,15.486356,16.879545,16.024958,15.400715,16.818878,15.967034,15.341793,16.776297,15.925804,15.300776,16.746685,15.897334,15.272053,16.725166,15.876816,15.2521105,16.708357,15.862035,15.237181,16.696539,15.851936,15.226998,16.689613,15.844313,15.21976,0.013392451,0.02354,0.021872982,0.01736985,0.024445537,0.039866067,0.05026544,0.08455789,0.122911856,0.005682776,0.0054258136,0.0049038026,0.005588406,0.005323561,0.0047632363,0.00543534,0.005177031,0.0046343105,0.0053025587,0.005040888,0.0045181247,0.005206092,0.0049438984,0.0044374703,0.0051483097,0.0048758644,0.0043834248,0.0051032505,0.0048264163,0.004344248,0.0050705094,0.004792021,0.004319857,0.005054293,0.004765629,0.0043044444,16.65942,0.34037337,0.2978292,3.0239506,0.07928849,15.820566,0.2640441,0.34559453,3.3033552,0.003811298,15.195567,0.29826432,0.3001073,3.2333765,0.011723555,0,-1
3,SGA-2020 3,PGC1310416,1310416,202.54443750000002,6.9345944,Sc,159.26,0.4017908,0.7816278,0.073888786,23.498482,16.85,False,LEDA-20181114,1,PGC1310416,1,True,202.544437500000

In [52]:
h1_mass[:5]

AGCNr,Name,RAdeg_HI,DECdeg_HI,RAdeg_OC,DECdeg_OC,Vhelio,W50,sigW,W20,HIflux,sigflux,SNR,RMS,Dist,sigDist,logMH,siglogMH,HIcode
int64,str8,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64
105367,........,0.00167,5.44333,0.00333,5.4425,11983,274,39,281,1.14,0.08,8.1,1.91,166.0,2.3,9.87,0.05,1
333313,........,0.00375,24.90889,359.9975,24.9075,11181,313,20,333,1.8,0.09,11.3,2.02,154.8,2.3,10.01,0.05,1
331060,478-009b,0.01042,23.08472,0.01417,23.0875,4463,160,4,184,1.96,0.07,14.7,2.35,50.6,10.4,9.07,0.18,1
331061,456-013.,0.01042,15.87222,0.00875,15.88167,6007,260,45,268,1.13,0.09,6.5,2.4,85.2,2.4,9.29,0.06,1
104570,........,0.00667,32.70834,0.005,32.71028,10614,245,6,250,0.86,0.07,6.6,1.86,147.0,2.3,9.64,0.06,1


In [48]:
tf_mastertable[tf_mastertable['SGA_ID']==202236 ]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,Z_CENTER,ROTCURVE_CHI2,VMAX,VMAX_ERR,RTURN,RTURN_ERR,ALPHA,ALPHA_ERR,INCLINATION,VROT_R26,VROT_R26_ERR,M_R26,M_R26_ERR,VROT_3P5R26,VROT_3P5R26_ERR,M_3P5R26,M_3P5R26_ERR,RABS_MAG26,M_STELLAR_CIGALE,M_STELLAR_CIGALE_ERR,M_STELLAR_PROVABGS,M_STELLAR_PROVABGS_ERR
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1070919447478281,218.467126790284,35.6021357274572,10517,sv3,0.689349774676162,0.000105899659336298,4,1.1599750071764,fuji/healpix/sv3/bright/105/10517/redrock-sv3-bright-10517.fits,TFT,202236,218.46712679028371,35.60213572745723,1.0,0.010328013447839866,0.9999999757533122,1.0,0.0,196230.58895498002,31.761552886604264,0.0347949861264779,--,--,--,--,--,--,--,--,--,0.0,--,--,--,0.0,--,--,-21.09168685298143,--,--,--,--
1070919447478283,218.473204372212,35.6069798702523,10517,sv3,0.0342643027523248,5.81042710273245e-06,0,521.161364927888,fuji/healpix/sv3/bright/105/10517/redrock-sv3-bright-10517.fits,TFT,202236,218.47320437221234,35.60697987025231,1.0,0.003408244437794044,0.32999999199926,1.0,0.0,-159.09487315708984,1.9748830179274115,0.0347949861264779,61.50510020149073,561.3099492572,157.08378064380133,0.08537294408848492,0.12780472231685672,0.36383992423415007,0.057502354761352084,1.4711405903507528,391.1285338274364,410.0510414676205,11.825310198144829,0.49090734012908754,444.0025751658789,718.8914816193749,12.479510220455634,0.6271846376573009,-21.09168685298143,9.759396229228562,--,--,--
39632991256840767,218.476198078587,35.6093656791547,10517,sv3,0.0347951441075021,4.38416406810812e-06,0,3660.84922169894,fuji/healpix/sv3/bright/105/10517/redrock-sv3-bright-10517.fits,TFT,202236,218.47619807858737,35.60936567915472,1.0,0.0,0.0,1.0,0.0,0.04736151956227686,1.6103905603883806,0.0347949861264779,61.50510020149073,561.3099492572,157.08378064380133,0.08537294408848492,0.12780472231685672,0.36383992423415007,0.057502354761352084,1.4711405903507528,391.1285338274364,410.0510414676205,11.825310198144829,0.49090734012908754,444.0025751658789,718.8914816193749,12.479510220455634,0.6271846376573009,-21.09168685298143,9.759396229228562,--,--,--
1070919447478282,218.470120134196,35.6045216866714,10517,sv3,0.035417316436115,3.76541421700099e-05,0,9.22480585216545,fuji/healpix/sv3/bright/105/10517/redrock-sv3-bright-10517.fits,TFT,202236,218.47012013419638,35.604521686671355,1.0,0.006919769010041057,0.6699999837535908,1.0,0.0,186.56993321400878,11.32671500875773,0.0347949861264779,61.50510020149073,561.3099492572,157.08378064380133,0.08537294408848492,0.12780472231685672,0.36383992423415007,0.057502354761352084,1.4711405903507528,391.1285338274364,410.0510414676205,11.825310198144829,0.49090734012908754,444.0025751658789,718.8914816193749,12.479510220455634,0.6271846376573009,-21.09168685298143,9.759396229228562,--,--,--
1070919447478284,218.47919196352,35.6117514140064,10517,sv3,0.0352543507475819,2.11623843739911e-05,0,39.4392753937282,fuji/healpix/sv3/bright/105/10517/redrock-sv3-bright-10517.fits,TFT,202236,218.47919196352018,35.61175141400644,1.0,0.003408244437787266,0.3299999919986037,1.0,0.0,137.71404887900843,6.412199866829514,0.0347949861264779,61.50510020149073,561.3099492572,157.08378064380133,0.08537294408848492,0.12780472231685672,0.36383992423415007,0.057502354761352084,1.4711405903507528,391.1285338274364,410.0510414676205,11.825310198144829,0.49090734012908754,444.0025751658789,718.8914816193749,12.479510220455634,0.6271846376573009,-21.09168685298143,9.759396229228562,--,--,--
1084113587011596,218.47919196352,35.6117514140064,10517,sv3,0.0352300902375281,1.03763672224515e-0

In [63]:
sga_ids = [9769, 11977, 12892, 14501, 16383, 19235, 19278, 25532, 30149, 31220, 32484, 36904, 37543, 46327, 58617, 73111, 88963, 91660, 90791, 106536, 
           114405, 114489, 122580, 126630, 125611, 128163, 128411, 130766, 149502, 153504, 175220, 176736, 184044, 186239, 189685, 188525, 190308, 
           192158, 196470, 199038, 202236, 205028, 209429, 212382, 218773, 235556, 254532, 272164, 272562, 276638, 288715, 289743, 291766, 300446, 
           302062, 305831, 309378, 319942, 330678, 349027, 349346, 363344, 373692, 378842, 384420, 389177, 393190, 395070, 398443, 398532, 416531, 
           431242, 430161, 432294, 450689, 450566, 455486, 457147, 458798, 462867, 466872, 467582, 471327, 469868, 471893, 473436, 477805, 481956, 
           485800, 493017, 507499, 526710, 544419, 544810, 544799, 544943, 548724, 548848, 550458, 564440, 569298, 568664, 570086, 582248, 591927, 
           607528, 608015, 629144, 630029, 630390, 635043, 638386, 641578, 643231, 643845, 644940, 647966, 649753, 649960, 651096, 654469, 659599, 
           663463, 671301, 671533, 694525, 696291, 696345, 706579, 720480, 722142, 726854, 728327, 729726, 729754, 736449, 742167, 743030, 744650, 
           752683, 757313, 757166, 758123, 759261, 759003, 766672, 783693, 793633, 804643, 805225, 819754, 820598, 823301, 824225, 826809, 827949, 
           833961, 833981, 843065, 843126, 845052, 887433, 901663, 901980, 908818, 908978, 911046, 914975, 914909, 916161, 920806, 923400, 925923, 
           925354, 928810, 928876, 932984, 947709, 953644, 957521, 957850, 967351, 968864, 971635, 975775, 972260, 977015, 976853, 979168, 978708, 
           989040, 989244, 1001302, 1003319, 1008726, 1014959, 1018187, 1021462, 1026141, 1031898, 1033651, 1033344, 1050173, 1065602, 1073430, 
           1086934, 1087039, 1087374, 1090772, 1091806, 1096564, 1098043, 1103477, 1111824, 1117018, 1124386, 1143335, 1146354, 1161160, 1167691, 
           1171841, 1179377, 1184894, 1184912, 1186034, 1187240, 1188032, 1188617, 1188926, 1193015, 1191665, 1194588, 1194958, 1199020, 1199539, 
           1201319, 1203786, 1204237, 1206707, 1218166, 1219319, 1254154, 1264056, 1264591, 1267967, 1271477, 1271549, 1272700, 1273605, 1278144, 
           1281619, 1282407, 1285273, 1286710, 1289613, 1293967, 1301768, 1304908, 1335916, 1338866, 1345100, 1348956, 1349555, 1354266, 1355575, 
           1361363, 1361399, 1365519, 1365549, 1367137, 1368018, 1368443, 1374964, 1379143, 1405053, 1429680, 1434487]

In [ ]:
#new column in tf mastertable for H1 masses
tf_mastertable['M_HI']=np.nan
tf_mastertable['M_HI_ERR']=np.nan

In [67]:
#find h1 mass matching RAs and DECs
h1_table_ra = h1_mass[h1_mass['HIcode']==1]['RAdeg_HI']
h1_table_dec = h1_mass[h1_mass['HIcode']==1]['DECdeg_HI']
for sga_id in sga_ids: 
    sga = SGA[SGA['SGA_ID'] == sga_id]
    h1_vals = h1_mass[(sga['RA'] <= (h1_table_ra + 0.01)) & (sga['RA'] >= (h1_table_ra - 0.01)) & 
                      (sga['DEC'] <= (h1_table_dec + 0.01)) & (sga['DEC'] >= (h1_table_dec - 0.01))]['logMH']
    tf_mastertable['M_H1'][np.logical_and(tf_mastertable['SGA_ID'] == sga_id, tf_mastertable['M_R26'] > 0)] = h1_mass

IndexError: boolean index did not match indexed array along dimension 0; dimension is 31502 but corresponding boolean dimension is 25434

In [ ]:
# put target ids in list if mass exists
sgas = []
sga_ids = tf_mastertable['SGA_ID'][tf_mastertable['M_R26'] > 0]

# find matching target ids for the first table
for sga_id in sga_ids:
    if len(h1_table[h1_table['SGA_ID'] == sga_id]) > 0:
        targets.append(sga_id)

#extract H1 data and update to mastertable
for sga_id in sga_ids:
    h1_mass = h1_table[h1_table['SGA_ID'] == sga_id]['LOGM'][0] #ask what columns are h1 mass!!!
    h1_mass_err = h1_table[h1_table['SGA_ID'] == sga_id]['LOGM_ERR'][0] #ask what column is h1 error
    tf_mastertable['M_H1'][np.logical_and(tf_mastertable['SGA_ID'] == sga_id, tf_mastertable['M_R26'] > 0)] = h1_mass
    tf_mastertable['M_H1_ERR'][np.logical_and(tf_mastertable['SGA_ID'] == sga_id, tf_mastertable['M_R26'] > 0)] = h1_mass_err